# Introducción a LightGBM

Este notebook cubre:
- ¿Qué es LightGBM?
- Ventajas y características
- Instalación y configuración
- Uso básico de LightGBM
- Parámetros principales
- Comparación con otros algoritmos

## 1. ¿Qué es LightGBM?

**LightGBM** (Light Gradient Boosting Machine) es un framework de gradient boosting desarrollado por Microsoft.

### Características principales:
- **Rápido**: Optimizado para velocidad y eficiencia de memoria
- **Escalable**: Maneja grandes volúmenes de datos
- **Preciso**: Alto rendimiento en tareas de machine learning
- **Soporte nativo**: Maneja valores categóricos sin necesidad de one-hot encoding
- **Múltiples tareas**: Clasificación, regresión, ranking

### Ventajas sobre otros algoritmos:
1. **Velocidad**: Más rápido que XGBoost y otros algoritmos de boosting
2. **Memoria**: Uso eficiente de memoria
3. **Precisión**: Mejor rendimiento en muchos casos
4. **Paralelización**: Soporte para GPU y procesamiento paralelo
5. **Manejo de datos grandes**: Optimizado para datasets grandes

## 2. Instalación

In [ ]:
# Instalar LightGBM (descomenta si no está instalado)
# !pip install lightgbm

import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('seaborn-v0_8')

print(f"LightGBM versión: {lgb.__version__}")
print("Librerías importadas correctamente")

## 3. Conceptos Fundamentales de LightGBM

### 3.1 Gradient Boosting

LightGBM utiliza **Gradient Boosting**, que es una técnica de ensemble learning:

1. **Boosting**: Combina múltiples modelos débiles (árboles) en un modelo fuerte
2. **Gradient**: Utiliza el gradiente de la función de pérdida para entrenar cada árbol
3. **Secuencial**: Cada árbol se entrena para corregir los errores del anterior

### 3.2 Diferencias Clave: Leaf-wise vs Level-wise

- **XGBoost/otros**: Usan level-wise (crecen nivel por nivel)
- **LightGBM**: Usa leaf-wise (crece hoja por hoja, eligiendo la hoja con mayor ganancia)

**Ventaja**: Leaf-wise puede lograr menor pérdida con el mismo número de hojas, pero requiere más cuidado con el overfitting.

## 4. Ejemplo Básico con Datos Simulados

In [ ]:
# Crear datos de ejemplo
from sklearn.datasets import make_classification

# Generar dataset sintético
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=4,
    n_clusters_per_class=1,
    random_state=42
)

df_ejemplo = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
df_ejemplo['target'] = y

print(f"Forma del dataset: {df_ejemplo.shape}")
print(f"\nDistribución de clases:")
print(df_ejemplo['target'].value_counts().sort_index())
print(f"\nPrimeras filas:")
print(df_ejemplo.head())

## 5. Entrenamiento Básico de LightGBM

In [ ]:
# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Datos de entrenamiento: {X_train.shape}")
print(f"Datos de prueba: {X_test.shape}")

In [ ]:
# Crear Dataset de LightGBM
# LightGBM requiere un formato especial de Dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

print("Datasets de LightGBM creados correctamente")

In [ ]:
# Parámetros básicos de LightGBM
params = {
    'objective': 'multiclass',  # Para clasificación multiclase
    'num_class': 4,  # Número de clases
    'metric': 'multi_logloss',  # Métrica de evaluación
    'boosting_type': 'gbdt',  # Gradient Boosting Decision Tree
    'num_leaves': 31,  # Número de hojas (importante para controlar complejidad)
    'learning_rate': 0.05,  # Tasa de aprendizaje
    'feature_fraction': 0.9,  # Fracción de características a usar en cada árbol
    'bagging_fraction': 0.8,  # Fracción de datos a usar en cada árbol
    'bagging_freq': 5,  # Frecuencia de bagging
    'verbose': 0  # Silenciar output
}

print("Parámetros configurados:")
for key, value in params.items():
    print(f"  {key}: {value}")

In [ ]:
# Entrenar el modelo
model = lgb.train(
    params,
    train_data,
    num_boost_round=100,  # Número de iteraciones (árboles)
    valid_sets=[train_data, test_data],
    valid_names=['train', 'eval'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=10),  # Parar si no mejora en 10 rondas
        lgb.log_evaluation(period=10)  # Mostrar métricas cada 10 rondas
    ]
)

print("\nModelo entrenado correctamente")

## 6. Predicciones y Evaluación

In [ ]:
# Hacer predicciones
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_class = np.argmax(y_pred, axis=1)  # Convertir probabilidades a clases

# Evaluar
accuracy = accuracy_score(y_test, y_pred_class)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_class))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_class)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - LightGBM')
plt.ylabel('Verdadero')
plt.xlabel('Predicho')
plt.show()

## 7. Parámetros Principales de LightGBM

### 7.1 Parámetros de Control

| Parámetro | Descripción | Valores Típicos |
|-----------|-------------|-----------------|
| `objective` | Función objetivo | 'multiclass', 'binary', 'regression' |
| `metric` | Métrica de evaluación | 'multi_logloss', 'binary_logloss', 'rmse' |
| `boosting_type` | Tipo de boosting | 'gbdt', 'dart', 'goss', 'rf' |
| `num_leaves` | Número de hojas | 31 (default), típicamente 2^max_depth - 1 |
| `learning_rate` | Tasa de aprendizaje | 0.01 - 0.3 |
| `num_iterations` | Número de árboles | 100 - 10000 |

### 7.2 Parámetros de Regularización

| Parámetro | Descripción | Efecto |
|-----------|-------------|--------|
| `lambda_l1` | Regularización L1 | Reduce overfitting, hace el modelo más simple |
| `lambda_l2` | Regularización L2 | Reduce overfitting |
| `min_gain_to_split` | Ganancia mínima para dividir | Previene divisiones inútiles |
| `min_data_in_leaf` | Mínimo de datos en hoja | Previene overfitting |

### 7.3 Parámetros de Sampling

| Parámetro | Descripción |
|-----------|-------------|
| `feature_fraction` | Fracción de características a usar (0-1) |
| `bagging_fraction` | Fracción de datos a usar (0-1) |
| `bagging_freq` | Frecuencia de bagging (cada N iteraciones) |

## 8. Importancia de Características

In [ ]:
# Obtener importancia de características
feature_importance = pd.DataFrame({
    'feature': [f'feature_{i}' for i in range(X.shape[1])],
    'importance': model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False)

print("Top 10 características más importantes:")
print(feature_importance.head(10))

# Visualizar
plt.figure(figsize=(10, 8))
sns.barplot(data=feature_importance.head(15), x='importance', y='feature')
plt.title('Importancia de Características - LightGBM')
plt.xlabel('Importancia (Gain)')
plt.tight_layout()
plt.show()

## 9. Visualización del Modelo

In [ ]:
# Visualizar un árbol (primer árbol)
ax = lgb.plot_tree(model, tree_index=0, figsize=(20, 12))
plt.title('Primer Árbol del Modelo LightGBM')
plt.show()

print("Nota: LightGBM crea muchos árboles. Este es solo el primero.")

In [ ]:
# Gráfico de importancia de características (método alternativo)
lgb.plot_importance(model, max_num_features=15, figsize=(10, 8))
plt.title('Importancia de Características - LightGBM')
plt.tight_layout()
plt.show()

## 10. Comparación con Scikit-learn API

LightGBM también tiene una API compatible con scikit-learn, más fácil de usar:

In [ ]:
# Usando la API de scikit-learn (más simple)
from lightgbm import LGBMClassifier

# Crear modelo
lgbm_classifier = LGBMClassifier(
    objective='multiclass',
    num_class=4,
    num_leaves=31,
    learning_rate=0.05,
    n_estimators=100,
    random_state=42,
    verbose=-1
)

# Entrenar
lgbm_classifier.fit(X_train, y_train)

# Predecir
y_pred_sklearn = lgbm_classifier.predict(X_test)

# Evaluar
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Accuracy (scikit-learn API): {accuracy_sklearn:.4f}")

print("\nVentajas de la API de scikit-learn:")
print("- Más familiar para usuarios de scikit-learn")
print("- Compatible con pipelines y GridSearchCV")
print("- Más simple de usar")

## 11. Tuning de Hiperparámetros Básico

In [ ]:
# Ejemplo de búsqueda de hiperparámetros básica
from sklearn.model_selection import GridSearchCV

# Definir grid de parámetros
param_grid = {
    'num_leaves': [15, 31, 50],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 200]
}

# Crear modelo base
lgbm_base = LGBMClassifier(
    objective='multiclass',
    num_class=4,
    random_state=42,
    verbose=-1
)

# Grid Search (comentado porque puede tardar)
# grid_search = GridSearchCV(
#     lgbm_base,
#     param_grid,
#     cv=3,
#     scoring='accuracy',
#     n_jobs=-1,
#     verbose=1
# )
# 
# grid_search.fit(X_train, y_train)
# print(f"Mejores parámetros: {grid_search.best_params_}")
# print(f"Mejor score: {grid_search.best_score_:.4f}")

print("Grid Search definido (descomenta para ejecutar)")

## 12. Ventajas y Cuándo Usar LightGBM

### Ventajas:
1. ✅ **Velocidad**: Muy rápido, especialmente en datasets grandes
2. ✅ **Memoria**: Uso eficiente de memoria
3. ✅ **Precisión**: Alto rendimiento en muchos problemas
4. ✅ **Manejo de categóricas**: Soporte nativo sin one-hot encoding
5. ✅ **GPU**: Soporte para aceleración por GPU

### Cuándo usar LightGBM:
- ✅ Datasets grandes (>10,000 filas)
- ✅ Muchas características
- ✅ Necesitas velocidad
- ✅ Problemas de clasificación o regresión
- ✅ Datos con características categóricas

### Cuándo NO usar LightGBM:
- ❌ Datasets muy pequeños (<1,000 filas) - puede overfittear fácilmente
- ❌ Necesitas interpretabilidad detallada (usa árboles simples)
- ❌ Datos con muchas características irrelevantes (usa feature selection primero)

## Resumen

En este notebook hemos aprendido:
1. ✅ ¿Qué es LightGBM y sus ventajas
2. ✅ Instalación y configuración básica
3. ✅ Entrenamiento de modelos con LightGBM
4. ✅ Parámetros principales y su significado
5. ✅ Importancia de características
6. ✅ API de scikit-learn vs API nativa
7. ✅ Cuándo usar LightGBM

**Próximos pasos:** En el siguiente notebook aplicaremos LightGBM al problema de predicción de fallas con datos reales.